# Experiments ECAI24 on the no2 dataset

7 input features, all continuous. No known distribution shifts in this dataset, so it is used to mimic the daily model updates.



In [1]:
# Basics
import numpy as np
import pandas as pd
import csv

# sci-kit learn
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.model_selection import RandomizedSearchCV
from util_scripts.convert import extract_sklearn_params, custom_nn_model

from joblib import dump, load
import gurobipy

pd.options.display.max_columns = 100
pd.options.display.max_rows = 150

import warnings
warnings.filterwarnings('ignore')

from util_scripts.preprocessor import Preprocessor, min_max_scale
from util_scripts.utilexp import *
from interval import *
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
tf.random.set_seed(1)
np.random.seed(1)

CYAN_COL = '\033[96m'
BLUE_COL = '\033[94m'
RED_COL = '\033[91m'
GREEN_COL = '\033[92m'
YELLOW_COL = '\033[93m'
RESET_COL = '\033[0m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'

In [2]:
df = pd.read_csv("./datasets/no2/no2.csv")
df = df.dropna()
df = df.replace(to_replace={'N': 0, 'P': 1})
ordinal_features = {}
discrete_features = {}
continuous_features = list(df.columns)[:-1]
CLASS = "binaryClass"

# min max scale
min_vals = np.min(df[continuous_features], axis=0)
max_vals = np.max(df[continuous_features], axis=0)
df_mm = min_max_scale(df, continuous_features, min_vals, max_vals)
columns = list(df_mm.columns)
display(df_mm)
# get X, y
X, y = df_mm.drop(columns=['binaryClass']), pd.DataFrame(df_mm['binaryClass'])

SPLIT = .2
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=SPLIT, shuffle=True,
                                                    random_state=0)
feat_var_map = {}
for i in range(len(X.columns)):
    feat_var_map[i] = [i]

,no2_concentration,cars_per_hour,temperature_at_2m,wind_speed,temperature_diff_2m_25m,wind_direction,hour_of_day,binaryClass
0,0.482404,0.844284,0.700252,0.468750,0.546392,0.202801,0.826087,0
1,0.362831,0.846118,0.629723,0.333333,0.525773,0.151261,0.565217,1
2,0.404232,0.162280,0.375315,0.062500,0.546392,0.782353,0.130435,0
3,0.611930,0.669122,0.287154,0.145833,0.680412,0.201681,0.956522,1
4,0.603835,0.803270,0.435768,0.239583,0.546392,0.176471,0.434783,1
...,...,...,...,...,...,...,...,...
495,0.596692,0.842110,0.556675,0.489583,0.453608,0.212885,0.434783,1
496,0.332732,0.569073,0.707809,0.645833,0.474227,0.582633,0.391304,1
497,0.570573,0.860087,0.599496,0.447917,0.474227,0.593838,0.565217,1
498,0.334757,0.393904,0.680101,0.020833,0.288660,0.298319,0.260870,0


In [4]:
np.random.seed(4)
idx_1 = np.sort(np.random.choice(range(500), 250))
idx_2 = np.array([i for i in range(500) if i not in idx_1])

X1 = pd.DataFrame(data=X.values[idx_1], columns=X.columns)
y1 = pd.DataFrame(data=y.values[idx_1], columns=y.columns)
X2 = pd.DataFrame(data=X.values[idx_2], columns=X.columns)
y2 = pd.DataFrame(data=y.values[idx_2], columns=y.columns)

SPLIT = .2
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, stratify=y1, test_size=SPLIT, shuffle=True,
                                                        random_state=0)


# Train models and Observe model shifts


In [5]:
# Randomdised search + 5-fold cross validation (default)
nn = MLPClassifier(learning_rate='adaptive', random_state=0)

# parameters
max_iter_vals = [int(i) for i in np.linspace(1000, 10000, 10)]
hidden_layer_sizes_vals = [(i) for i in range(5, 25)]
batch_size_vals = [8, 16, 32, 64]
learning_rate_init_vals = [0.001, 0.002, 0.005, 0.01, 0.02, 0.05]

#distributions = dict(max_iter=max_iter_vals, hidden_layer_sizes=hidden_layer_sizes_vals)
distributions = dict(hidden_layer_sizes=hidden_layer_sizes_vals,
                     batch_size=batch_size_vals,
                     learning_rate_init=learning_rate_init_vals,
                     max_iter=max_iter_vals, )

#nns = RandomizedSearchCV(nn, distributions, scoring='f1_macro')
#nns = RandomizedSearchCV(nn, distributions, scoring='accuracy')
#search = nns.fit(X, y)
#print(search.best_params_)


In [6]:
clf = MLPClassifier(learning_rate='adaptive', hidden_layer_sizes=16, learning_rate_init=0.005, batch_size=8,
                    max_iter=2000, random_state=0)

# 5-fold cross validation
scoring = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']
scores = cross_validate(clf, X, y, scoring=scoring)
for name in list(scores.keys()):
    if name == 'fit_time' or name == 'score_time':
        continue
    print("%0.2f %s with a std of %0.2f" % (scores[name].mean(), name, scores[name].std()))

clf = MLPClassifier(learning_rate='adaptive', hidden_layer_sizes=16, learning_rate_init=0.005, batch_size=8,
                    max_iter=2000, random_state=0)

clf.fit(X1_train, y1_train)
resres = clf.predict(X1_test.values)
print('\n', classification_report(y1_test, resres, target_names=[f'bad credit (0)', f'good credit (1)'], digits=3))
resres = clf.predict(X1_train.values)
print('\n', classification_report(y1_train, resres, target_names=[f'bad credit (0)', f'good credit (1)'], digits=3))


0.61 test_accuracy with a std of 0.04
0.62 test_precision_macro with a std of 0.05
0.61 test_recall_macro with a std of 0.04
0.61 test_f1_macro with a std of 0.04

                  precision    recall  f1-score   support

 bad credit (0)      0.700     0.538     0.609        26
good credit (1)      0.600     0.750     0.667        24

       accuracy                          0.640        50
      macro avg      0.650     0.644     0.638        50
   weighted avg      0.652     0.640     0.637        50


                  precision    recall  f1-score   support

 bad credit (0)      0.785     0.716     0.749       102
good credit (1)      0.729     0.796     0.761        98

       accuracy                          0.755       200
      macro avg      0.757     0.756     0.755       200
   weighted avg      0.758     0.755     0.755       200



In [7]:
dump(clf, 'no2.joblib')

['no2.joblib']

# Experiments: computing counterfactuals

#### Procedures

These procedures are covered by UtilExp class

1. Train M on D1
2. Get delta-min, build M+ and M-: incrementally train M 5 times, using different 10% of D2 each time, then get the maximum inf-distance between the incremented models and M. Construct M+ and M- using delta-min
3. Get M2: incrementally train M on D2
4. Select test instances: randomly select 50 D1 instances to explain, clf(x)=0, desired class=1
5. Report metrics using each baseline

#### Metrics
- Proximity: normalised L1: "Scaling Guarantees for Nearest CEs" page 7
- Sparsity: L0
- Validity-delta: percentage of test instances that 1) have counterfactuals valid on m1, 2) counterfactuals valid on M+ and M- under delta_min
- Validity-m2: percentage of test instances that 1) have counterfactual(s), 2) these counterfactual(s) are all valid on both m1 and m2
- LOF: average LOF score

In [8]:
clf = load("no2.joblib")
gurobipy.setParam("FeasibilityTol", 1e-03)
gurobipy.setParam("OptimalityTol", 1e-03)
gurobipy.setParam("IntFeasTol", 1e-03)


Restricted license - for non-production use only - expires 2025-11-24
Set parameter FeasibilityTol to value 0.001
Set parameter OptimalityTol to value 0.001
Set parameter IntFeasTol to value 0.001


In [9]:
util_exp = UtilExp(clf, X1, y1, X2, y2, columns, ordinal_features, discrete_features, continuous_features, feat_var_map, num_test_instances=200, gap=0.15)
print(util_exp.delta_max)
print(util_exp.delta_min)

0.11449453866860582
0.020924421229148754


In [10]:
# save model trained on the whole dataset
m2 = copy.deepcopy(clf)
m2.partial_fit(X2, y2)
util_exp.Mmax = m2

In [11]:
# pre-verification on the points soundness 
valids = util_exp.verify_soundness()
print(len(valids))

percentage of sound model changes: 0.4424778761061947
50


In [12]:
valids = util_exp.verify_soundness(update_test_instances=True)

percentage of sound model changes: 0.4424778761061947
test instances updated to sound (x, Delta) pairs, length: 50


Compute the $\delta$ difference between m1 and m2

In [13]:
input_size, n_layers, output_size, output_act, h_act, optimizer, params = extract_sklearn_params(clf)
tf_model = custom_nn_model(input_size, n_layers, params, output_size, h_act, output_act, optimizer)
     
# Set model weights
for k, v in params.items():
	tf_model.layers[k].set_weights(v)
        
tf_model.save('./models/no2.h5', save_format='h5')   

input_size, n_layers, output_size, output_act, h_act, optimizer, params = extract_sklearn_params(m2)
tf_model = custom_nn_model(input_size, n_layers, params, output_size, h_act, output_act, optimizer)
     
# Set model weights
for k, v in params.items():
	tf_model.layers[k].set_weights(v)
	
tf_model.save('./models/no2_retrained.h5', save_format='h5')  

original_model = tf.keras.models.load_model('./models/no2.h5', compile=False)
old_weights = {}
for l in range(1,len(original_model.layers)):
	old_weights[l] = original_model.layers[l].get_weights()


model_retrained = tf.keras.models.load_model('./models/no2_retrained.h5', compile=False)
retrained_weights = {}
for l in range(1,len(model_retrained.layers)):
	retrained_weights[l] = model_retrained.layers[l].get_weights()


max_diff = -1
for l in range(1,len(old_weights)):
	old_layer_weights = old_weights[l][0]
	new_retrained_weights = retrained_weights[l][0]

	difference = abs(old_layer_weights - new_retrained_weights)
	
	for list_weights in difference:
		max_distance = max(list_weights)
		if max_distance > max_diff:
			max_diff = max_distance

print("\nThe maximum distance between weights is:", max_diff)


The maximum distance between weights is: 0.07160799


### CFX computation: in the following cells we both compute the CFX using MILP and the proposed probabilistic APΔS approach. 

In [14]:
# OURS-ROBUST: compute CFX based on the n sound points discovered above
ours_robust_ces_apas = util_exp.run_ours_robust(approx=True)
util_exp.evaluate_ces(ours_robust_ces_apas)
cfxs_robust_apas = ours_robust_ces_apas
print(len(cfxs_robust_apas))

50it [00:10,  4.67it/s]

total computation time in s: 10.735352039337158
found: 1.0
average normalised L1: 0.04218988369320217
average normalised L0: 0.2116399999999999
average lof score: 1.0
counterfactual validity: 1.0
delta validity: 0.0
m2 validity: 1.0
50


In [15]:
# OURS-ROBUST: compute CFX based on the n sound points discovered above
ours_robust_ces = util_exp.run_ours_robust()
util_exp.evaluate_ces(ours_robust_ces)
cfxs_robust = ours_robust_ces
print(len(cfxs_robust))

50it [00:03, 14.29it/s]


total computation time in s: 3.5024325847625732
found: 1.0
average normalised L1: 0.05878411805181261
average normalised L0: 0.23737999999999992
average lof score: 1.0
counterfactual validity: 1.0
delta validity: 1.0
m2 validity: 1.0
50


Testing how many robust CFX remains robust after the retraining (i.e., expanding the $\delta$)

In [16]:
# double checking the robustness of the CFXs found by APΔS approach after retraining
tot_robust_cfx = len(cfxs_robust_apas)
robust_cfx_after_retrain = 0

for cfx in cfxs_robust_apas:
    if model_retrained(np.array(cfx.reshape(1,-1))) >= 0.5:
        robust_cfx_after_retrain += 1

print(f"Percentage CFXs robust after the retraing: {(robust_cfx_after_retrain/tot_robust_cfx)*100}%") 

Percentage CFXs robust after the retraing: 100.0%


In [17]:
tot_robust_cfx = len(cfxs_robust)
robust_cfx_after_retrain = 0

for cfx in cfxs_robust:
    if model_retrained(np.array(cfx.reshape(1,-1))) >= 0.5:
        robust_cfx_after_retrain += 1

print(f"Percentage CFXs robust after the retraing: {(robust_cfx_after_retrain/tot_robust_cfx)*100}%") 

Percentage CFXs robust after the retraing: 100.0%


## Experiment: probabilistic approximation approach to compute $\Delta$-robustness

In [19]:
def estimate_robustness(model, delta, cfx, concretizations, use_biases=True, robustness=True):

    """
    Utility method for the estimation of the CFX (not) Δ-robustness in the INN.

    Returns:
    --------
        rate: float
            estimation of the CFX (not) Δ-robustness computed with 'concretizations' models concretizations from the INN
    """
    np.random.seed(1)
    # Store initial weights
    old_weights = {}
    for l in range(1,len(model.layers)):
        old_weights[l] = model.layers[l].get_weights()

    for _ in range(concretizations):
        
        #perturbated_weights = {}
        input_features = np.array(cfx)

        for l in range(1,len(old_weights)+1):
            layer_weights = old_weights[l][0]
            if use_biases: layer_biases  = old_weights[l][1]
            
            weights_perturbation = np.random.uniform(-delta, delta, layer_weights.shape)
            if use_biases: biases_perturbation = np.random.uniform(-delta, delta, layer_biases.shape)
           
            
            layer_weights = [layer_weights+weights_perturbation]

            if use_biases: 
                layer_biases = [layer_biases+biases_perturbation]
                preactivated_res = np.dot(input_features, layer_weights) + layer_biases
            else:
                preactivated_res = np.dot(input_features, layer_weights)

            if l != len(old_weights):
                #relu
                activated_res = np.maximum(0.0, preactivated_res)
            else:
                #sigmoid
                activated_res = 1/(1 + np.exp(-preactivated_res))
            
            input_features = activated_res
            
        if input_features < 0.5:
            return 0  
    
    return 1

def compute_delta_max_MILP(cfx, delta_init, verbose=False):
  
    lower = 1/(1 + np.exp(-util_exp.is_robust_custom_delta_new(cfx, delta_init)))
    if lower < 0.5: return 0 # CFX not robust
        
    delta = delta_init
    while lower >= 0.5: # over-approx lower bound is >= 0.5, i.e., x results robust
        delta = 2*delta
        lower = 1/(1 + np.exp(-util_exp.is_robust_custom_delta_new(cfx, delta)))
        if verbose: 
            print(f'Testing δ={delta}')
            print(f'Lower is: {lower}')
    
    delta_max = delta/2
    
    while True:
        if abs(delta-delta_max) < delta_init:
            return delta_max

        if verbose: print(f"\nInterval to test is: [{delta_max}, {delta}]")
        
        delta_new = (delta_max+delta)/2
        lower = 1/(1 + np.exp(-util_exp.is_robust_custom_delta_new(cfx, delta_new)))
        if verbose: 
            print(f'Testing δ={delta_new}')
            print(f'Rate is: {lower}')
        
        if lower >= 0.5:
            delta_max = delta_new
        else:
            delta = delta_new



def compute_delta_max(model, cfx, delta_init, concretizations, use_biases=True, verbose=False):
  
    rate = estimate_robustness(model, delta_init, cfx, concretizations, use_biases)
    if rate != 1: return 0 # CFX not robust
        
    delta = delta_init
    while rate == 1: # for all the concretizations x results robust
        delta = 2*delta
        rate = estimate_robustness(model, delta, cfx, concretizations, use_biases)
        if verbose: 
            print(f'Testing δ={delta}')
            print(f'Rate is: {rate}')
    
    delta_max = delta/2
    
    while True:
        if abs(delta-delta_max) < delta_init:
            return delta_max

        if verbose: print(f"\nInterval to test is: [{delta_max}, {delta}]")
        
        delta_new = (delta_max+delta)/2
        rate = estimate_robustness(model, delta_new, cfx, concretizations, use_biases)
        if verbose: 
            print(f'Testing δ={delta_new}')
            print(f'Rate is: {rate}')
        
        if rate == 1:
            delta_max = delta_new
        else:
            delta = delta_new
        

### Start the computation

In [20]:
model = tf.keras.models.load_model('./models/no2.h5', compile=False)
alpha = 0.999
R = 0.995
concretizations = int(np.emath.logn(R, (1-alpha)))
delta_init = 0.0001
delta_AAAI = 0.020924421229148754

with open("full_results_no2.csv", mode='w', newline='') as file:

    csv_writer = csv.writer(file)
    csv_writer.writerow(["CFX", "AAAI δ_max", "Wilks δ_max", "MILP δ_max", "Difference"])

    wilks = []
    milp = []
    print( f"{CYAN_COL}Condifence α =(1-R^n)={(1-R**concretizations)*100}%, R={R*100}%, Concretizations(n)={concretizations}{RESET_COL}")

    # start computing the new deltas with the approximation
    for cfx in cfxs_robust:

        delta_max_sampling = compute_delta_max(model, cfx.reshape(1,-1), delta_init, concretizations,verbose=False)
        delta_max_MILP = compute_delta_max_MILP(cfx, delta_init,verbose=False) 
        difference = abs(delta_max_sampling-delta_max_MILP)
        wilks.append(delta_max_sampling)
        milp.append(delta_max_MILP)
        csv_writer.writerow([cfx,  delta_AAAI, delta_max_sampling, delta_max_MILP, difference])

        print('δ_max =', delta_max_sampling)
        print('δ improvement w.r.t original MILP\'s δ is',difference)
        print("______________________________________________________________________________________")



    csv_writer.writerow([" "])
    csv_writer.writerow(["Mean MILP","Mean Wilks"])
    csv_writer.writerow([np.mean(np.array(milp)),np.mean(np.array(wilks))])


Condifence α =(1-R^n)=99.89995272421471%, R=99.5%, Concretizations(n)=1378
δ_max = 0.07800000000000001
δ improvement w.r.t original MILP's δ is 0.05560000000000001
______________________________________________________________________________________
δ_max = 0.082
δ improvement w.r.t original MILP's δ is 0.0598
______________________________________________________________________________________
δ_max = 0.08650000000000001
δ improvement w.r.t original MILP's δ is 0.0627
______________________________________________________________________________________
δ_max = 0.08195
δ improvement w.r.t original MILP's δ is 0.05904999999999999
______________________________________________________________________________________
δ_max = 0.07115000000000002
δ improvement w.r.t original MILP's δ is 0.04955000000000002
______________________________________________________________________________________
δ_max = 0.08715
δ improvement w.r.t original MILP's δ is 0.06295
________________________________